In [1]:
import copy
import os
import numpy as np
from numpy.linalg import norm
import matplotlib
import matplotlib.pyplot as plt
# plt.rcParams['animation.ffmpeg_path'] = "C:\\Users\\Fredric\\Documents\\ffmpeg-n4.4-latest-win64-gpl-4.4\\bin"
plt.rcParams['animation.ffmpeg_path'] = "C:\\Users\\Fredric\\Documents\\ffmpeg\\ffmpeg-n4.4-latest-win64-gpl-4.4\\bin\\ffmpeg.exe"

# Path for .pkl files
# target_dir = "C:\\Users\\Fredric\\Documents\\Volcano\\quail_volcano\\scenarios\\mixture_shocktube\\"
# target_dir = "C:\\Users\\Fredric\\Documents\\Volcano\\quail_volcano\\scenarios\\mixture_shocktube\\pilot_runs\\"
target_dir = "C:\\Users\\Fredric\\Documents\\Volcano\\quail_volcano\\scenarios\\mixture_shocktube\\dev_runs\\"
# Path for Quail source code
source_dir = "C:\\Users\\Fredric\\Documents\\Volcano\\quail_volcano\\src\\"

In [2]:
# Import quail modules
os.chdir(source_dir)
import argparse
import importlib
import sys

import defaultparams as default_deck
import errors
from general import ShapeType, SolverType, PhysicsType

import meshing.common as mesh_common
import meshing.gmsh as mesh_gmsh
import meshing.tools as mesh_tools

import numerics.helpers.helpers as helpers
import numerics.timestepping.tools as stepper_tools

import physics.zerodimensional.zerodimensional as zerod
import physics.euler.euler as euler
import physics.navierstokes.navierstokes as navierstokes
import physics.scalar.scalar as scalar
import physics.chemistry.chemistry as chemistry
import physics.multiphasevpT.multiphasevpT as multiphasevpT

import processing.readwritedatafiles as readwritedatafiles
import processing.post as post
import processing.plot as plot
import processing
import processing.mdtools as mdtools

import solver.DG as DG
import solver.ADERDG as ADERDG
import solver.tools as solver_tools

import time
import multiprocessing as mp
from multidomain import Domain, Observer

os.chdir(target_dir)

In [148]:
# Search energy, pressure
e = U_all[:,:,:,4].squeeze()
e_int_all = e - 0.5*rho_all*u_all**2
rho_R = solver.state_coeffs[-1:,-1:,0:3].sum(axis=2).squeeze()
e_int_R = solver.state_coeffs[:,:,4] - 0.5*solver.state_coeffs[:,:,3]**2 / \
  solver.state_coeffs[:,:,0:3].sum(axis=2)
e_int_R = e_int_R[-1:,-1:].squeeze()
p_R = 1e5
RH = lambda rho, e_int, p :\
  (rho_R * e_int - rho * e_int_R) / (rho - rho_R) -  0.5*(p + p_R)

plt.figure(5)
plt.clf()
plt.plot(RH(rho_all, e_int_all, p_all))

In [210]:
# Getting post-shock values
rho_post_shock = np.interp(0, RH(rho_all, e_int_all, p_all), rho_all)
u_post_shock = np.interp(0, RH(rho_all, e_int_all, p_all), u_all)
p_post_shock = np.interp(0, RH(rho_all, e_int_all, p_all), p_all)
# Capping
ind = np.interp(0, RH(rho_all, e_int_all, p_all), range(len(RH(rho_all, e_int_all, p_all))))
clipped_fan = lambda v: np.array([*v[0:int(ind)+1], 
  np.interp(0, RH(rho_all, e_int_all, p_all), v)])
plt.figure(5)
plt.clf()
plt.plot(clipped_fan(w_all), clipped_fan(u_all), '.')
# clipped_fan(w_all).shape
# np.interp(0, RH(rho_all, e_int_all, p_all), u_all)
# u_all[0:int(ind)+1][-1]

plt.subplot(3,3,1)
plt.plot(x_mesh, solver.state_coeffs[:,:,0:3].sum(axis=2).squeeze())
plt.plot(solver.time*extend(clipped_fan(w_all)), extend(clipped_fan(rho_all)), '.-')

plt.subplot(3,3,2)
plt.plot(x_mesh, solver.state_coeffs[:,:,0].squeeze())
plt.plot(solver.time*extend(clipped_fan(w_all)), extend(clipped_fan(U_all[:,:,:,0].squeeze())), '.-')

plt.subplot(3,3,3)
plt.plot(x_mesh, solver.state_coeffs[:,:,1].squeeze())
plt.plot(solver.time*extend(clipped_fan(w_all)), extend(clipped_fan(U_all[:,:,:,1].squeeze())), '.-')

plt.subplot(3,3,4)
plt.plot(x_mesh, solver.state_coeffs[:,:,2].squeeze())
plt.plot(solver.time*extend(clipped_fan(w_all)), extend(clipped_fan(U_all[:,:,:,2].squeeze())), '.-')

plt.subplot(3,3,5)
plt.plot(x_mesh, solver.state_coeffs[:,:,3].squeeze())
plt.plot(solver.time*extend(clipped_fan(w_all)), extend(clipped_fan(U_all[:,:,:,3].squeeze())), '.-')

plt.subplot(3,3,6)
u_mesh = (solver.state_coeffs[:,:,3] / solver.state_coeffs[:,:,0:3].sum(axis=2)).squeeze()
plt.plot(x_mesh, u_mesh)
plt.plot(solver.time*extend(clipped_fan(w_all)), extend(clipped_fan(u_all)), '.-')


In [156]:
# Adding in shock
shock_speed = u_post_shock + (p_post_shock - p_R) / (rho_post_shock * u_post_shock)
# extend_R = lambda v: np.array([*v[0:int(ind)+1], 
  # np.interp(0, RH(rho_all, e_int_all, p_all), v)])
# shock_speed * (rho_post_shock - rho_R) - rho_post_shock * u_post_shock
rho_post_shock

1536.7203343482313

In [266]:
''' Solving Rankine-Hugoniot + contact condition
There may not be a solution to this!?
'''
# Fan coord -> state
u_mapping = lambda xi: np.interp(xi, w_all, u_all)
p_mapping = lambda xi: np.interp(xi, w_all, p_all)

# Temperature mapping
def e_int(yA, yWv, yM, p, rho):
  # Compute temperature-independent vars
  rho_m = physics.Liquid["rho0"] * (1.0 + 1.0/physics.Liquid["K"] *
    (p - physics.Liquid["p0"]))
  alpha_m = yM * rho / rho_m
  alpha_g = 1.0 - alpha_m
  # Compute temperature from temperature-dependent vars
  T = (p * alpha_g) / (rho*(yA*physics.Gas[0]["R"] + yWv*physics.Gas[1]["R"]))
  alpha_A = rho*yA / p * physics.Gas[0]["R"] * T
  alpha_Wv = rho*yWv / p * physics.Gas[1]["R"] * T
  # Compute internal energy density
  e_int = rho * (yA * physics.Gas[0]["c_v"] * T
    + yWv * physics.Gas[1]["c_v"] * T 
    + yM * (physics.Liquid["c_m"] * T + physics.Liquid["E_m0"])
  )
  return e_int

U_R = solver.state_coeffs[-1,-1,:]
rho_R = solver.state_coeffs[-1:,-1:,0:3].sum(axis=2).squeeze()
yA_R = solver.state_coeffs[-1,-1,0] / rho_R
yWv_R = solver.state_coeffs[-1,-1,1] / rho_R
yM_R = solver.state_coeffs[-1,-1,2] / rho_R

e_int_mapping = lambda xi, rho: e_int(yA_R, yWv_R, yM_R, p_mapping(xi), rho)

# Define the residual equation for S = S_fixed, J1 = J1_fixed
def residual_eqn_contact(v):
    xi, rho = v
    eqn_1 = (rho_R / (rho - rho_R) * e_int_mapping(xi, rho) - \
      rho / (rho - rho_R) * e_int_R - 0.5*(p_mapping(xi) + p_R)) / p_R
    eqn_2 = (rho * u_mapping(xi)**2.0 * rho_R / (rho - rho_R) 
      - p_mapping(xi) + p_R) / p_R
    # print(e_int_mapping(xi, rho))
    # print(eqn_1, eqn_2)
    return (eqn_1, eqn_2)

# Compute invariant-constrained state
xi, rho = fsolve(residual_eqn_contact, (-100, 1.2))

# On success, compute post-contact state
u_contact = u_mapping(xi)
p_contact = p_mapping(xi)
U_contact_ess = np.array([rho*yA_R, rho*yWv_R, rho*yM_R, rho*u_contact, 
  e_int(yA_R, yWv_R, yM_R, p_contact, rho) + 0.5*rho*u_contact**2.0])

shock_speed = rho * u_contact / (rho - rho_R)

In [255]:
fan_x = fan_contact_x(w_all, xi)
# U_all[:,:,:,0].squeeze().shape
fan_q1 = np.array([
  *clipped_fan2(U_all[:,:,:,0].squeeze(), xi), clipped_fan2(U_all[:,:,:,0].squeeze(), xi)[-1],
  U_contact_ess[0], U_contact_ess[0]])
plt.figure()
plt.plot(fan_q1)

In [70]:
# Compute vorticity
plot_idx = 250

plt.clf()

domain_indices = [1, 2, 3]
for dom_idx in domain_indices:
  solver = readwritedatafiles.read_data_file(f"v0_phi0800m_atm{dom_idx}_{plot_idx}.pkl")
  sample_basis_phys_grad_elems = []
  sample_points_ref = solver.basis.PRINCIPAL_NODE_COORDS
  for elem_ID in range(solver.mesh.num_elems):
    solver.basis.get_basis_val_grads(
            sample_points_ref, # sample_points_in_domain[elem_ID,:,:],
            get_val=True,
            get_ref_grad=True,
            get_phys_grad=True,
            ijac=solver.elem_helpers.ijac_elems[elem_ID,0:sample_points_ref.shape[0],:,:])
    sample_basis_phys_grad_elems.append(solver.basis.basis_phys_grad)
  du_ij = np.einsum('ijnl, ink -> ijkl',
        np.array(sample_basis_phys_grad_elems),
        solver.state_coeffs[:,:,1:3]/solver.state_coeffs[:,:,0:1])
  vorticity = du_ij[:,:,1:2,0] - du_ij[:,:,0:1,1]
  x, _ = mdtools.downsample(solver)
  # mdtools.custom_plot_2D(x, vorticity, solver, levels=None)
  mdtools.plot_mean(x, np.log(np.abs(vorticity)), (-5,7))

clims = (-5,7)

cb = plt.colorbar(plt.cm.ScalarMappable(
      norm=matplotlib.colors.Normalize(vmin=clims[0], vmax=clims[1]),
      cmap=plt.get_cmap()))
cb.set_label("$log_{10} |\\omega|$")

In [ ]:
# Compute vorticity
plot_idx = 250

plt.clf()

domain_indices = [1, 2, 3]
for dom_idx in domain_indices:
  solver = readwritedatafiles.read_data_file(f"v0_phi0800m_atm{dom_idx}_{plot_idx}.pkl")
  sample_basis_phys_grad_elems = []
  sample_points_ref = solver.basis.PRINCIPAL_NODE_COORDS
  for elem_ID in range(solver.mesh.num_elems):
    solver.basis.get_basis_val_grads(
            sample_points_ref, # sample_points_in_domain[elem_ID,:,:],
            get_val=True,
            get_ref_grad=True,
            get_phys_grad=True,
            ijac=solver.elem_helpers.ijac_elems[elem_ID,0:sample_points_ref.shape[0],:,:])
    sample_basis_phys_grad_elems.append(solver.basis.basis_phys_grad)
  du_ij = np.einsum('ijnl, ink -> ijkl',
        np.array(sample_basis_phys_grad_elems),
        solver.state_coeffs[:,:,1:3]/solver.state_coeffs[:,:,0:1])
  vorticity = du_ij[:,:,1:2,0] - du_ij[:,:,0:1,1]
  x, _ = mdtools.downsample(solver)
  # mdtools.custom_plot_2D(x, vorticity, solver, levels=None)
  mdtools.plot_mean(x, np.log(np.abs(vorticity)), (-5,7))

clims = (-5,7)

cb = plt.colorbar(plt.cm.ScalarMappable(
      norm=matplotlib.colors.Normalize(vmin=clims[0], vmax=clims[1]),
      cmap=plt.get_cmap()))
cb.set_label("$log_{10} |\\omega|$")

In [102]:
# Compute sound speed
plot_idx = 250

plt.clf()

domain_indices = [1, 2, 3]
for dom_idx in domain_indices:
  solver = readwritedatafiles.read_data_file(f"v0_phi0800m_atm{dom_idx}_{plot_idx}.pkl")
  sample_basis_phys_grad_elems = []
  sample_points_ref = solver.basis.PRINCIPAL_NODE_COORDS
  x, soundspeed = mdtools.downsample(solver, "SoundSpeed")
  # mdtools.custom_plot_2D(x, vorticity, solver, levels=None)
  mdtools.plot_mean(x, soundspeed, (0,500))

clims = (0,500)

cb = plt.colorbar(plt.cm.ScalarMappable(
      norm=matplotlib.colors.Normalize(vmin=clims[0], vmax=clims[1]),
      cmap=plt.get_cmap()))
cb.set_label("sound speed (m/s)")

In [ ]:
plot_idx = 250

plt.clf()

domain_indices = [1, 2, 3]
for dom_idx in domain_indices:
  solver = readwritedatafiles.read_data_file(f"v0_phi0800m_atm{dom_idx}_{plot_idx}.pkl")
  sample_basis_phys_grad_elems = []
  sample_points_ref = solver.basis.PRINCIPAL_NODE_COORDS
  for elem_ID in range(solver.mesh.num_elems):
    solver.basis.get_basis_val_grads(
            sample_points_ref, # sample_points_in_domain[elem_ID,:,:],
            get_val=True,
            get_ref_grad=True,
            get_phys_grad=True,
            ijac=solver.elem_helpers.ijac_elems[elem_ID,0:sample_points_ref.shape[0],:,:])
    sample_basis_phys_grad_elems.append(solver.basis.basis_phys_grad)
  du_ij = np.einsum('ijnl, ink -> ijkl',
        np.array(sample_basis_phys_grad_elems),
        solver.state_coeffs[:,:,1:3]/solver.state_coeffs[:,:,0:1])
  normgradrho = np.linalg.norm(
    np.einsum('ijnl, ink -> ijkl',
      np.array(sample_basis_phys_grad_elems),
      solver.state_coeffs)[:,:,0],
    axis=2, keepdims=True)
  x, _ = mdtools.downsample(solver)
  # mdtools.custom_plot_2D(x, vorticity, solver, levels=None)
  mdtools.plot_mean(x, normgradrho, (1e-3,0.03))

clims = (1e-3,0.03)

cb = plt.colorbar(plt.cm.ScalarMappable(
      norm=matplotlib.colors.Normalize(vmin=clims[0], vmax=clims[1]),
      cmap=plt.get_cmap()))
cb.set_label("$|\\nabla \\alpha_{air} \\rho_{air}|$")

In [85]:
# Plot |grad p|
plot_idx = 250

plt.clf()

domain_indices = [1, 2, 3]
for dom_idx in domain_indices:
  solver = readwritedatafiles.read_data_file(f"v0_phi0800m_atm{dom_idx}_{plot_idx}.pkl")
  sample_basis_phys_grad_elems = []
  sample_points_ref = solver.basis.PRINCIPAL_NODE_COORDS
  for elem_ID in range(solver.mesh.num_elems):
    solver.basis.get_basis_val_grads(
            sample_points_ref, # sample_points_in_domain[elem_ID,:,:],
            get_val=True,
            get_ref_grad=True,
            get_phys_grad=True,
            ijac=solver.elem_helpers.ijac_elems[elem_ID,0:sample_points_ref.shape[0],:,:])
    sample_basis_phys_grad_elems.append(solver.basis.basis_phys_grad)
  # du_ij = np.einsum('ijnl, ink -> ijkl',
  #       np.array(sample_basis_phys_grad_elems),
  #       solver.physics.compute_variable("Pressure", solver.state_coeffs))
  normgradp = np.linalg.norm(
    np.einsum('ijnl, ink -> ijkl',
      np.array(sample_basis_phys_grad_elems),
      solver.physics.compute_variable("Pressure", solver.state_coeffs)),
    axis=2, keepdims=True)
  x, _ = mdtools.downsample(solver)
  # mdtools.custom_plot_2D(x, vorticity, solver, levels=None)
  mdtools.plot_mean(x, normgradp, (0,3250))#(0,25))

clims = (0,3250)

cb = plt.colorbar(plt.cm.ScalarMappable(
      norm=matplotlib.colors.Normalize(vmin=clims[0], vmax=clims[1]),
      cmap=plt.get_cmap()))
cb.set_label("$|\\nabla p|$")

In [ ]:
import ternary
%matplotlib tk

idx = 5
# solver_atm1 = readwritedatafiles.read_data_file(f"mixture_shocktube_atm1_{idx}.pkl")
# solver_atm2 = readwritedatafiles.read_data_file(f"mixture_shocktube_atm2_{idx}.pkl")
# solver_conduit1 = readwritedatafiles.read_data_file(f"mixture_shocktube_conduit_{idx}.pkl")
# solver_conduit2 = readwritedatafiles.read_data_file(f"mixture_shocktube_conduit2_{idx}.pkl")
solver_atm1 = readwritedatafiles.read_data_file(f"v0_phi0800m_atm1_{idx}.pkl")
solver_atm2 = readwritedatafiles.read_data_file(f"v0_phi0800m_atm2_{idx}.pkl")
solver_conduit1 = readwritedatafiles.read_data_file(f"v0_phi0800m_conduit1_{idx}.pkl")
solver_conduit2 = readwritedatafiles.read_data_file(f"v0_phi0800m_conduit2_{idx}.pkl")

fontsize = 12

def prep_ternary():
  ## Boundary and Gridlines
  scale = 100.0
  figure, tax = ternary.figure(scale=scale)

  # Draw Boundary and Gridlines
  tax.boundary(linewidth=2.0)
  tax.gridlines(color="black", multiple=10)
  tax.gridlines(color="blue", multiple=5, linewidth=0.1)

  # Set Axis labels and Title
  # Set ticks
  tax.ticks(axis='lbr', multiple=10, linewidth=2)

  ternary.plt.show()
  plt.box(False)

  # Remove default Matplotlib Axes
  tax.clear_matplotlib_ticks()
  return tax

def get_wt_percents(solver, coordset="AWvM"):
  rho = np.sum(solver.state_coeffs[:,:,solver.physics.get_mass_slice()], axis=2)
  if coordset == "AWvM":
    y1 = solver.state_coeffs[:,:,0] / rho * 100.0
    y2 = solver.state_coeffs[:,:,1] / rho * 100.0
    y3 = solver.state_coeffs[:,:,2] / rho * 100.0
    return list(zip(y1.ravel(), y2.ravel(), y3.ravel()))
  elif coordset == "AWtMd":
    y1 = solver.state_coeffs[:,:,0] / rho * 100.0
    # Water transform
    arhoWt = solver.state_coeffs[:,:,solver.physics.get_state_index("pDensityWt")]
    arhoWd = arhoWt - solver.state_coeffs[:,:,1]
    y2 = arhoWt / rho * 100.0
    y3 = (solver.state_coeffs[:,:,2]-arhoWd) / rho * 100.0
  else:
    raise KeyError("Invalid coordset.")
  return list(zip(y1.ravel(), y2.ravel(), y3.ravel()))

''' Air, exsolved water, magma mixture'''
tax = prep_ternary()
tax.bottom_axis_label("wt% air", fontsize=fontsize)
tax.right_axis_label("wt% water, exsolved", fontsize=fontsize)
tax.left_axis_label("wt% frac melt + crystals + water, dissolved", fontsize=fontsize)

tax.right_corner_label("Air", fontsize=fontsize)
tax.top_corner_label("Exsolved water", fontsize=fontsize)
tax.left_corner_label("Magma", fontsize=fontsize)

tax.scatter(get_wt_percents(solver_atm1, "AWvM"), color='blue')
tax.scatter(get_wt_percents(solver_atm2, "AWvM"), color='red')
tax.scatter(get_wt_percents(solver_conduit1, "AWvM"), color='magenta')
tax.scatter(get_wt_percents(solver_conduit2, "AWvM"), color='cyan')
# tax.set_title("", fontsize=fontsize)

tax = prep_ternary()
tax.bottom_axis_label("wt% air", fontsize=fontsize)
tax.right_axis_label("wt% water, total", fontsize=fontsize)
tax.left_axis_label("wt% frac melt + crystals", fontsize=fontsize)

tax.right_corner_label("Air", fontsize=fontsize)
tax.top_corner_label("Total water", fontsize=fontsize)
tax.left_corner_label("Dry magma", fontsize=fontsize)

tax.scatter(get_wt_percents(solver_atm1, "AWtMd"), marker='d', color='blue', label="atm near")
tax.scatter(get_wt_percents(solver_atm2, "AWtMd"), marker='d', color='brown', label="atm far")
tax.scatter(get_wt_percents(solver_conduit1, "AWtMd"), marker='s', color='magenta', label="conduit upper")
tax.scatter(get_wt_percents(solver_conduit2, "AWtMd"), marker='s',color='cyan', label="conduit lower")

tax.legend()

In [3]:
def compute_eig_aco_negative(U, physics):
  ''' Computes eigenvector corresponding to u-c'''

  '''Compute rows of flux Jacobian for tracer states'''
  # Row of flux Jacobian for tracer states
  rho = U[:,:,physics.get_mass_slice()].sum(axis=2)
  u = U[:,:,physics.get_state_index("XMomentum")] / rho
  # Truncated row of flux Jacobian for tracer states
  #   b = [-u, -u, -u, 1, 0] / rho * q_i
  # where q_i is the partial density of the given tracer state
  N_states_hyperbolic = 5
  N_states_tracer = 2
  # Fill temporary construction vector with size [ne, nq, ns_hyp]
  b_sub = np.tile(np.zeros_like(u), (1,1,N_states_hyperbolic))
  b_sub[:,:,physics.get_mass_slice()] = -u/rho
  b_sub[:,:,physics.get_state_index("XMomentum")] = 1.0/rho
  # Fill temporary construction vector with size [ne, nq, ns_tracer]
  slice_like_tracers = (physics.get_state_index("pDensityWt"), 
    physics.get_state_index("pDensityC"))
  arho_tracers = U[:,:,slice_like_tracers]
  # Compute rows of flux Jacobian for tracer states
  b = np.einsum("ijk, ijl -> ijkl", arho_tracers, b_sub)

  ''' Compute u-c eigenvector of hyperbolic subsystem '''
  # Size [ne, nq, ns_hyp]
  #   x = y1 y2 y3 u-a H - au
  # Mass fractions
  y1 = U[:,:,0] / rho
  y2 = U[:,:,1] / rho
  y3 = U[:,:,2] / rho
  eigvec_hyp = np.zeros_like(b_sub)
  H = physics.compute_additional_variable("TotalEnthalpy", U, True)
  a = physics.compute_additional_variable("SoundSpeed", U, True)
  eigvec_hyp[:,:,0] = y1
  eigvec_hyp[:,:,1] = y2
  eigvec_hyp[:,:,2] = y3
  eigvec_hyp[:,:,3] = u - a
  eigvec_hyp[:,:,4] = H - a*u

  ''' Compute extension of the hyperbolic subsystem acoustic eigenvector'''
  # Compute (b^T * eigvec) / (eigval - u) -- 
  eigvec_ext = np.einsum("ijkl, ijl -> ijk", b, eigvec_hyp) / (-a)
  return np.concatenate((eigvec_hyp, eigvec_ext), axis=2)

In [5]:
# Constructing pressure boundary condition using damped-Newton-RK
# solver = solver_conduit1
plot_idx = 250
solver_atm1_init = readwritedatafiles.read_data_file(f"v0_phi0800m_conduit1_{plot_idx}.pkl")
solver = solver_atm1_init
physics = solver.physics
bgroup = solver_atm1_init.mesh.boundary_groups["x2"]
U_init = solver.state_coeffs[solver.bface_helpers.elem_IDs[solver.mesh.boundary_groups["x2"].number], :, :]
# Modify initial to lower velocity
U = U_init.copy()
# Remove kinetic
U[:,:,4] -= 0.5 * U[:,:,3]**2 / U[:,:,0:3].sum(axis=2)
# Reduce momentum
U[:,:,3] /= 3.0
# Add back kinetic
U[:,:,4] += 0.5 * U[:,:,3]**2 / U[:,:,0:3].sum(axis=2)

# Define numerics
p_target = 1.6e6*2
damping_idx_scale = 100
additional_damping_factor = 1
N_steps_max = 100
# Permissive step count
N_steps_max = np.max([N_steps_max, 10*damping_idx_scale])
p_rel_tol = 1e-6

p = 1e-15
p_last = 1e15
stuck_counter = 0

p_fn = lambda U : solver.physics.compute_additional_variable("Pressure", U, True)
dpdU_fn = lambda U : solver.physics.compute_pressure_sgradient(U)
f = lambda U: compute_eig_aco_negative(U, solver.physics)

# Observation
p_hist = []
S_hist = []
y1_hist = []
y2_hist = []
y3_hist = []
yWt_hist = []
yC_hist = []
aux_hist = []
rho_hist = []
M_hist = []

for i in range(N_steps_max):
    ''' Set damped Newton step size '''
    # Discrete Gaussian damping
    damping = 1.0 - np.exp(-((i+1)/damping_idx_scale)**2)
    damping *= additional_damping_factor
    # Compute Newton step size
    newton_step_size = (p_target - p_fn(U)) / \
        np.einsum("ijk, ijk -> ij", dpdU_fn(U), f(U))
    
    # Check for stalling
    # if (p - p_last) / p_target < p_rel_tol and i > 2 * damping_idx_scale:
    #     # Increase damping (high damping for monotonic approach to target)
    #     # (Deterministic alternative to stochastic perturbation)
    #     damping *= (0.9)**(stuck_counter+1)
    #     stuck_counter += 1
    p_last = p

    # Butcher table for Cash-Karp RK quadrature
    B = np.array([[1/5, 0, 0, 0, 0],
         [3/40, 9/40, 0, 0, 0],
         [3/10, -9/10, 6/5, 0, 0],
         [-11/54, 5/2, -70/27, 35/27, 0],
         [1631/55296, 175/512, 575/13824, 44275/110592, 253/4096]])
    w = np.array([37/378, 0, 250/621, 125/594 , 0, 512/1771])
    # Compute damped step size for ODE integration
    damped_step_size = damping * newton_step_size
    # RK step 0
    num_stages = B.shape[0] + 1
    k = np.zeros(tuple(np.append([num_stages], list(U.shape))))
    k[0,:,:,:] = f(U)
    for j in range(B.shape[0]):
        k[j+1,:,:,:]= f(U + damped_step_size*
            np.einsum("m, mijk -> ijk", B[j,0:j+1], k[0:j+1,:]))
    U += damped_step_size * np.einsum("i, ijkl -> jkl", w, k)
    # U += damped_step_size * f(U)
    
    # Compute pressure
    p = p_fn(U)

    # q_hist(1+i,:) = q;
    p_hist.append(p)
    
    rho = U[:,:,0:3].sum(axis=2)
    y1 = U[:,:,0] / rho
    y2 = U[:,:,1] / rho
    y3 = U[:,:,2] / rho
    rhoA = U[:,:,0]/physics.compute_additional_variable("volFracA", U, True)
    rhoWv = U[:,:,1]/physics.compute_additional_variable("volFracWv", U, True)
    T = physics.compute_additional_variable("Temperature", U, True)
    S = y1 * solver.physics.Gas[0]["c_v"] * np.log(p / rhoA**solver.physics.Gas[0]["gamma"]) + \
        y2 * solver.physics.Gas[1]["c_v"] * np.log(p / rhoWv**solver.physics.Gas[1]["gamma"]) + \
        y3 * solver.physics.Liquid["c_m"] * np.log(T)
    
    a = solver.physics.compute_additional_variable("SoundSpeed", U, True)
    beta2 = solver.physics.compute_additional_variable("beta", U, True)
    velx = U[:,:,3] / rho
    # aux = U[:,:,3] / rho + 2 * a / \
      # (solver.physics.compute_additional_variable("Gamma", U, True) - 1)
    aux = U[:,:,3] / rho + a / \
      solver.physics.compute_additional_variable("beta", U, True)
    Mx = velx / a

    y1_hist.append(y1.squeeze())
    y2_hist.append(y2.squeeze())
    y3_hist.append(y3.squeeze())
    yWt_hist.append((U[:,:,5]/rho).squeeze())
    yC_hist.append((U[:,:,6]/rho).squeeze())
    rho_hist.append(rho.squeeze())
    S_hist.append(S.squeeze())
    aux_hist.append(aux.squeeze())
    M_hist.append(Mx.squeeze())
    # u2g1c_hist(1+i,1) = q(2) / q(1) ...
    #     + 2 / (gamma-1) * a(p_bind(q),q(1));

    
    #  Check termination tolerance
    if np.abs(p_target - p) / p_target < p_rel_tol:
        break


# solver.physics.compute_pressure_sgradient()


# solver.bface_helpers.x_bgroups[bgroup.number] # [ne_bdry, nq_bdry, ndims]
# quad_wts = solver_atm1_init.bface_helpers.quad_wts # [nq_bdry, 1]
# normals = solver_atm1_init.bface_helpers.normals_bgroups[bgroup.number] # [ne_bdry, nq_bdry, ndims]

# # solver_atm1_init.
# Uq_synthetic = np.ones_like(solver_atm1_init.bface_helpers.normals_bgroups[bgroup.number])[:,:,0:1] # [ne_bdry, nq_bdry, ns]
# # weighted_Uq = np.einsum('ijk, jm -> ijk', Uq_synthetic, quad_wts)

# bgroup = solver_atm1_init.mesh.boundary_groups["x2"]
# normals = solver_atm1_init.bface_helpers.normals_bgroups[bgroup.number]
# # Compute boundary measure
# boundary_length = solver_atm1_init.bface_helpers.face_lengths_bgroups[bgroup.number].sum()
# # Compute average
# U_avg = np.einsum('ijm, ijk, jn -> k',
#   np.linalg.norm(normals, axis=2, keepdims=True),
#   Uq_synthetic,
#   solver_atm1_init.bface_helpers.quad_wts) / boundary_length

# # U_avg = U_avg/boundary_length
# U_avg

In [36]:
# Constructing inlet boundary condition using damped-Newton-RK

plot_idx = 250
solver_atm1_init = readwritedatafiles.read_data_file(f"v0_phi0800m_conduit1_{plot_idx}.pkl")
solver = solver_atm1_init
physics = solver.physics
bgroup = solver_atm1_init.mesh.boundary_groups["x1"]
U_init = solver.state_coeffs[solver.bface_helpers.elem_IDs[solver.mesh.boundary_groups["x1"].number], :, :]

# Add non zero momentum
# Remove kinetic
U_init[:,:,4] -= 0.5 * U_init[:,:,3]**2 / U_init[:,:,0:3].sum(axis=2)
# Increase momentum
U_init[:,:,3] = U_init[:,:,0:3].sum(axis=2) * 100.0
# Add back kinetic
U_init[:,:,4] += 0.5 * U_init[:,:,3]**2 / U_init[:,:,0:3].sum(axis=2)

# Define magma chamber state
U_mc = np.zeros_like(U)
U_mc[:,:,0] = 0.0
U_mc[:,:,1] = 0.0
U_mc[:,:,2] = 3100.0
U_mc[:,:,3] = 0.0 # Momentum: replaced
U_mc[:,:,4] = U[:,:,4]*1
U_mc[:,:,5] = 0.0
U_mc[:,:,6] = 300.0    # Crystallinity

# Define numerics
damping_idx_scale = 5
additional_damping_factor = 1
N_steps_max = 25
# Permissive step count
N_steps_max = np.max([N_steps_max, 10*damping_idx_scale])
p_rel_tol = 1e-6

p = 1e-15
p_last = 1e15
stuck_counter = 0

p_fn = lambda U : solver.physics.compute_additional_variable("Pressure", U, True)
dpdU_fn = lambda U : solver.physics.compute_pressure_sgradient(U)
f = lambda U: compute_eig_aco_negative(U, solver.physics)

# Observation
p_hist = []
S_hist = []
y1_hist = []
y2_hist = []
y3_hist = []
yWt_hist = []
yC_hist = []
aux_hist = []
rho_hist = []
M_hist = []
H_hist = []

U = U_mc.copy()

for i in range(N_steps_max):
    ''' Set damped Newton step size '''
    # Discrete Gaussian damping
    damping = 1.0 - np.exp(-((i+1)/damping_idx_scale)**2)
    damping *= additional_damping_factor
    # Compute Newton step size
    # newton_step_size = (p_target - p_fn(U)) / \
    #     np.einsum("ijk, ijk -> ij", dpdU_fn(U), f(U))
    # Check sonic orthogonality (eigenvector orth. to objective gradient, i.e.,
    # is sonic). Dot prod is r_i * grad_q objective = r_i * grad_q (rho*u)
    dot_prod = f(U)[:,:,3]
    if np.abs(dot_prod.squeeze()) < 1e-10:
      break
    # Compute momentum Newton step size
    newton_step_size = (U_init[:,:,3] - U[:,:,3]) / dot_prod
    
    # Check for stalling
    # if (p - p_last) / p_target < p_rel_tol and i > 2 * damping_idx_scale:
    #     # Increase damping (high damping for monotonic approach to target)
    #     # (Deterministic alternative to stochastic perturbation)
    #     damping *= (0.9)**(stuck_counter+1)
    #     stuck_counter += 1
    p_last = p

    # Butcher table for Cash-Karp RK quadrature
    B = np.array([[1/5, 0, 0, 0, 0],
         [3/40, 9/40, 0, 0, 0],
         [3/10, -9/10, 6/5, 0, 0],
         [-11/54, 5/2, -70/27, 35/27, 0],
         [1631/55296, 175/512, 575/13824, 44275/110592, 253/4096]])
    w = np.array([37/378, 0, 250/621, 125/594 , 0, 512/1771])
    # Compute damped step size for ODE integration
    damped_step_size = damping * newton_step_size
    # RK step 0
    num_stages = B.shape[0] + 1
    k = np.zeros(tuple(np.append([num_stages], list(U.shape))))
    k[0,:,:,:] = f(U)
    for j in range(B.shape[0]):
        k[j+1,:,:,:]= f(U + damped_step_size*
            np.einsum("m, mijk -> ijk", B[j,0:j+1], k[0:j+1,:]))
    U += damped_step_size * np.einsum("i, ijkl -> jkl", w, k)
    # U += damped_step_size * f(U)
    
    # Compute momentum
    p = U[:,:,3]

    # q_hist(1+i,:) = q;
    p_hist.append(p)
    
    rho = U[:,:,0:3].sum(axis=2)
    y1 = U[:,:,0] / rho
    y2 = U[:,:,1] / rho
    y3 = U[:,:,2] / rho
    rhoA = U[:,:,0]/physics.compute_additional_variable("volFracA", U, True)
    rhoWv = U[:,:,1]/physics.compute_additional_variable("volFracWv", U, True)
    T = physics.compute_additional_variable("Temperature", U, True)
    S = y1 * solver.physics.Gas[0]["c_v"] * np.log(p / rhoA**solver.physics.Gas[0]["gamma"]) + \
        y2 * solver.physics.Gas[1]["c_v"] * np.log(p / rhoWv**solver.physics.Gas[1]["gamma"]) + \
        y3 * solver.physics.Liquid["c_m"] * np.log(T)
    
    a = solver.physics.compute_additional_variable("SoundSpeed", U, True)
    beta2 = solver.physics.compute_additional_variable("beta", U, True)
    velx = U[:,:,3] / rho
    # aux = U[:,:,3] / rho + 2 * a / \
      # (solver.physics.compute_additional_variable("Gamma", U, True) - 1)
    aux = U[:,:,3] / rho + a / \
      solver.physics.compute_additional_variable("beta", U, True)
    Mx = velx / a
    H = (U[:,:,4] + solver.physics.compute_additional_variable("Pressure", U, True))/rho

    y1_hist.append(y1.squeeze())
    y2_hist.append(y2.squeeze())
    y3_hist.append(y3.squeeze())
    yWt_hist.append((U[:,:,5]/rho).squeeze())
    yC_hist.append((U[:,:,6]/rho).squeeze())
    rho_hist.append(rho.squeeze())
    S_hist.append(S.squeeze())
    aux_hist.append(aux.squeeze())
    M_hist.append(Mx.squeeze())
    H_hist.append(H.squeeze())
    # u2g1c_hist(1+i,1) = q(2) / q(1) ...
    #     + 2 / (gamma-1) * a(p_bind(q),q(1));

    
    #  Check termination tolerance
    if np.abs(U_init[:,:,3] - U[:,:,3]) / U_init[:,:,3] < p_rel_tol:
        break
    else:
      print(f"relerr:{np.abs(U_init[:,:,3] - U[:,:,3]) / U_init[:,:,3]}")


# solver.physics.compute_pressure_sgradient()


# solver.bface_helpers.x_bgroups[bgroup.number] # [ne_bdry, nq_bdry, ndims]
# quad_wts = solver_atm1_init.bface_helpers.quad_wts # [nq_bdry, 1]
# normals = solver_atm1_init.bface_helpers.normals_bgroups[bgroup.number] # [ne_bdry, nq_bdry, ndims]

# # solver_atm1_init.
# Uq_synthetic = np.ones_like(solver_atm1_init.bface_helpers.normals_bgroups[bgroup.number])[:,:,0:1] # [ne_bdry, nq_bdry, ns]
# # weighted_Uq = np.einsum('ijk, jm -> ijk', Uq_synthetic, quad_wts)

# bgroup = solver_atm1_init.mesh.boundary_groups["x2"]
# normals = solver_atm1_init.bface_helpers.normals_bgroups[bgroup.number]
# # Compute boundary measure
# boundary_length = solver_atm1_init.bface_helpers.face_lengths_bgroups[bgroup.number].sum()
# # Compute average
# U_avg = np.einsum('ijm, ijk, jn -> k',
#   np.linalg.norm(normals, axis=2, keepdims=True),
#   Uq_synthetic,
#   solver_atm1_init.bface_helpers.quad_wts) / boundary_length

# # U_avg = U_avg/boundary_length
# U_avg

relerr:[[0.960818044045802]]
relerr:[[0.819132748817984]]
relerr:[[0.57265707700908]]
relerr:[[0.3033934517773]]
relerr:[[0.112355414387359]]
relerr:[[0.026771913658855]]
relerr:[[0.003782081127877]]
relerr:[[0.000292627395939]]
relerr:[[1.14620820110135e-05]]


C:\Users\Fredric\AppData\Local\Temp\ipykernel_52816\1820175723.py:112: RuntimeWarning: invalid value encountered in true_divide
  rhoA = U[:,:,0]/physics.compute_additional_variable("volFracA", U, True)
C:\Users\Fredric\AppData\Local\Temp\ipykernel_52816\1820175723.py:113: RuntimeWarning: invalid value encountered in true_divide
  rhoWv = U[:,:,1]/physics.compute_additional_variable("volFracWv", U, True)
C:\Users\Fredric\AppData\Local\Temp\ipykernel_52816\1820175723.py:124: RuntimeWarning: divide by zero encountered in true_divide
  aux = U[:,:,3] / rho + a / \
